In [1]:
import numpy as np

In [2]:
def gaussian_kernel(x1,x2,noise,length):
    n1 = len(x1)
    n2 = len(x2)
    kernel = np.zeros(n1,n2)
    for i in range(n1):
        for j in range(n2):
            kernel[i,j] = noise^2*exp(-0.5*((x1[i]-x2[j])/length)^2)
    return kernel


In [ ]:
def LLH_GP(x,y,m,noise,length):
    ker = Gaussian_kernel(x,x, sigma, lambda)
    return(-1/2*(log(det(ker))+t((y-m))%*%solve(ker)%*%(y-m)))

